#Document retrieval from wikipedia data

#Fire up GraphLab Create

In [2]:
import graphlab

#Load some text data - from wikipedia, pages on people

In [3]:
people = graphlab.SFrame('people_wiki.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\zcj\AppData\Local\Temp\graphlab_server_1506416783.log.0


This non-commercial license of GraphLab Create for academic use is assigned to zhangchaojie@ruc.edu.cn and will expire on September 24, 2018.


Data contains:  link to wikipedia article, name of person, text of article.

In [4]:
people.head()

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


In [5]:
len(people)

59071

#Explore the dataset and checkout the text it contains

##Exploring the entry for president Obama

In [6]:
obama = people[people['name'] == 'Barack Obama']

In [42]:
obama

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 1

Data:
+-------------------------------+--------------+-------------------------------+
|              URI              |     name     |              text             |
+-------------------------------+--------------+-------------------------------+
| <http://dbpedia.org/resour... | Barack Obama | barack hussein obama ii br... |
+-------------------------------+--------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_count          |             tfidf             |
+-------------------------------+-------------------------------+
| {'operations': 1L, 'repres... | {'operations': 3.811771079... |
+-------------------------------+-------------------------------+
[1 rows x 5 columns]

##Exploring the entry for actor George Clooney

#Get the word counts for Obama article

In [10]:
obama['word_count'] = graphlab.text_analytics.count_words(obama['text'])

##Sort the word counts for the Obama article

###Turning dictonary of word counts into a table

In [12]:
obama_word_count_table = obama[['word_count']].stack('word_count', new_column_name = ['word','count'])

###Sorting the word counts to show most common words at the top

In [13]:
obama_word_count_table.head()

word,count
normalize,1
sought,1
combat,1
continued,1
unconstitutional,1
8,1
californias,1
1996,1
marriage,1
defense,1


In [14]:
obama_word_count_table.sort('count',ascending=False)

word,count
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
a,7
he,7


Most common words include uninformative words like "the", "in", "and",...

#Compute TF-IDF for the corpus 

To give more weight to informative words, we weigh them by their TF-IDF scores.

In [15]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])
people.head()

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'since': 1L, 'carltons':1L, 'being': 1L, '2005': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1L, 'thomas':1L, 'closely': 1L, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1L, 'issued':1L, 'mainly': 1L, ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1L,'bauforschung': 1L, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'legendary': 1L,'gangstergenka': 1L, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'now': 1L, 'currently':1L, 'less': 1L, 'being': ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2L,'producer': 1L, 'tribe': ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1L, 'salon': 1L,'gangs': 1L, 'being': ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1L,'frankie': 1L, 'labels': ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1L,'deborash': 1L, ..."


In [20]:
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

In [26]:
people['tfidf'] = tfidf

In [27]:
people.head()

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'since': 1L, 'carltons':1L, 'being': 1L, '2005': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1L, 'thomas':1L, 'closely': 1L, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1L, 'issued':1L, 'mainly': 1L, ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1L,'bauforschung': 1L, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'legendary': 1L,'gangstergenka': 1L, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'now': 1L, 'currently':1L, 'less': 1L, 'being': ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2L,'producer': 1L, 'tribe': ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1L, 'salon': 1L,'gangs': 1L, 'being': ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1L,'frankie': 1L, 'labels': ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1L,'deborash': 1L, ..."


##Examine the TF-IDF for the Obama article

In [28]:
obama = people[people['name'] == 'Barack Obama']

In [29]:
obama[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


Words with highest TF-IDF are much more informative.

#Manually compute distances between a few people

Let's manually compare the distances between the articles for a few famous people.  

In [30]:
clinton = people[people['name'] == 'Bill Clinton']

In [84]:
beckham = people[people['name'] == 'David Beckham']

##Is Obama closer to Clinton than to Beckham?

We will use cosine distance, which is given by

(1-cosine_similarity) 

and find that the article about president Obama is closer to the one about former president Clinton than that of footballer David Beckham.

In [85]:
victoria=people[people['name']=='Victoria Beckham']

In [32]:
graphlab.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])

0.8339854936884276

In [33]:
graphlab.distances.cosine(obama['tfidf'][0],beckham['tfidf'][0])

0.9791305844747478


#Build a nearest neighbor model for document retrieval

We now create a nearest-neighbors model and apply it to document retrieval.  

In [82]:
knn_model = graphlab.nearest_neighbors.create(people,features=['tfidf'],label='name')

Starting brute force nearest neighbors model training.

#Applying the nearest-neighbors model for retrieval

##Who is closest to Obama?

In [83]:
knn_model.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 10.028ms     |

| Done         |         | 100         | 295.174ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,David Beckham,0.0,1
0,Steven Gerrard,0.741610738255,2
0,Didier Drogba,0.747747747748,3
0,Gordon Strachan,0.75,4
0,Wayne Rooney,0.752351097179,5


As we can see, president Obama's article is closest to the one about his vice-president Biden, and those of other politicians.  

##Other examples of document retrieval

In [36]:
swift = people[people['name'] == 'Taylor Swift']

In [37]:
knn_model.query(swift)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 15.628ms     |

| Done         |         | 100         | 343.651ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Taylor Swift,0.0,1
0,Carrie Underwood,0.76231884058,2
0,Alicia Keys,0.764705882353,3
0,Jordin Sparks,0.769633507853,4
0,Leona Lewis,0.776119402985,5


In [38]:
jolie = people[people['name'] == 'Angelina Jolie']

In [39]:
knn_model.query(jolie)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 15.631ms     |

| Done         |         | 100         | 323.407ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Angelina Jolie,0.0,1
0,Brad Pitt,0.784023668639,2
0,Julianne Moore,0.795857988166,3
0,Billy Bob Thornton,0.803069053708,4
0,George Clooney,0.8046875,5


In [40]:
arnold = people[people['name'] == 'Arnold Schwarzenegger']

In [41]:
knn_model.query(arnold)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 16.012ms     |

| Done         |         | 100         | 540.002ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Arnold Schwarzenegger,0.0,1
0,Jesse Ventura,0.818918918919,2
0,John Kitzhaber,0.824615384615,3
0,Lincoln Chafee,0.833876221498,4
0,Anthony Foxx,0.833910034602,5


In [43]:
Elton=people[people['name']=='Elton John']

In [44]:
Elton

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: Unknown

Data:
+-------------------------------+------------+-------------------------------+
|              URI              |    name    |              text             |
+-------------------------------+------------+-------------------------------+
| <http://dbpedia.org/resour... | Elton John | sir elton hercules john cb... |
+-------------------------------+------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_count          |             tfidf             |
+-------------------------------+-------------------------------+
| {'all': 1L, 'six': 1L, 'pr... | {'all': 1.6431112434912472... |
+-------------------------------+-------------------------------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [51]:
Elton[['word_count']].stack('word_count',new_column_name=['word','count']).sort('count',ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
he,7
john,7
on,6
since,5


In [48]:
type(Elton[['word_count']])

graphlab.data_structures.sframe.SFrame

In [52]:
Elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
overallelton,10.9864953892
tonightcandle,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


In [63]:
Elton['tfidf'][0]

{'1': 4.195753163848633,
 '10': 2.3157231098806563,
 '100': 8.922437249692202,
 '1947': 3.756656611073944,
 '1967': 3.01368460510379,
 '19702000': 10.293348208665249,
 '1976': 2.71622627778857,
 '1980s': 2.9688582293167167,
 '1988': 2.4491074905234376,
 '1992': 2.278351314316948,
 '1994': 2.2375557576874794,
 '1996': 2.135691193468776,
 '1997': 4.259668904415891,
 '1998': 2.0687826320938068,
 '2': 2.2482408126990716,
 '200': 3.8524016680323285,
 '2002': 1.8753125887822302,
 '2004': 3.3806887216718016,
 '2005': 1.6425861253275964,
 '2008': 1.5093391374786154,
 '2012he': 6.27696518791286,
 '2013': 1.9545642372230505,
 '2014': 2.2073995783446634,
 '21': 5.594501726978586,
 '25': 2.7469019349192254,
 '27': 3.110616229728885,
 '30': 2.6266580085851636,
 '300': 4.553555296486015,
 '31': 3.294838566414648,
 '33': 4.654993539331503,
 '40': 6.981906890681876,
 '49': 5.309741586956912,
 '58': 5.437419304329975,
 'a': 0.05619184472583146,
 'abbey': 5.693190564500702,
 'about': 1.9265128988375737,

In [66]:
beckham['tfidf'][0]

{'1': 2.0978765819243166,
 '100': 5.9482914997948,
 '115': 6.2329051981188295,
 '12': 2.511583590065563,
 '17': 2.7543211528412543,
 '18': 2.726778428203672,
 '1975': 2.762331876587333,
 '1992': 2.278351314316948,
 '1996': 2.135691193468776,
 '1998': 2.0687826320938068,
 '1999': 3.9936033767292685,
 '2': 2.2482408126990716,
 '2000': 1.8763068991994527,
 '2002': 1.8753125887822302,
 '2004': 1.6903443608359008,
 '2004renowned': 10.986495389225194,
 '2006': 1.520737905384506,
 '2007': 1.4879730697555795,
 '2009': 1.5644364836042695,
 '2010': 1.5928339601219734,
 '201213': 5.620519374203343,
 '2013': 3.909128474446101,
 '2014': 2.2073995783446634,
 '2016': 6.174311033852777,
 '2017': 7.348909229498808,
 '20year': 6.979162203992724,
 '21': 2.797250863489293,
 '50': 3.3613882409862943,
 '58': 5.437419304329975,
 'a': 0.01685755341774944,
 'ability': 4.213415013569659,
 'age': 2.138848033513307,
 'aged': 4.748170764185686,
 'among': 2.299897032948229,
 'an': 0.8947172672456913,
 'and': 0.0089

In [86]:
graphlab.distances.cosine(victoria['tfidf'][0],Elton['tfidf'][0])

0.9567006376655429

In [68]:
paul=people[people['name']=='Paul McCartney']

In [69]:
paul[['tfidf']].stack('tfidf',new_column_name=['word','count'])

word,count
children,2.42528531239
five,2.21373017928
married,2.32797626172
father,2.53865227594
education,2.4487155642
land,3.95652247752
hunting,6.16621382362
rights,3.0667756283
subjects,4.35713213579
charities,5.40676556324


In [70]:
paul

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: Unknown

Data:
+-------------------------------+----------------+
|              URI              |      name      |
+-------------------------------+----------------+
| <http://dbpedia.org/resour... | Paul McCartney |
+-------------------------------+----------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| sir james paul mccartney m... | {'all': 1L, 'gold': 1L, 'o... |
+-------------------------------+-------------------------------+
+-------------------------------+
|             tfidf             |
+-------------------------------+
| {'all': 1.6431112434912472... |
+-------------------------------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [71]:
graphlab.distances.cosine(paul['tfidf'][0],Elton['tfidf'][0])

0.8250310029221779

In [88]:
knn_word_model=graphlab.nearest_neighbors.create(people,features=['word_count'],label='name')

Starting brute force nearest neighbors model training.

In [87]:
knn_model.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 12.031ms     |

| Done         |         | 100         | 313.099ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,0.0,1
0,Phil Collins,0.76399026764,2
0,Rod Stewart,0.773333333333,3
0,Annie Lennox,0.776623376623,4
0,Barry Gibb,0.780952380952,5


In [89]:
knn_word_model.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 8.022ms      |

| Done         |         | 100         | 260.22ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,0.0,1
0,Phil Collins,0.76399026764,2
0,Rod Stewart,0.773333333333,3
0,Annie Lennox,0.776623376623,4
0,Barry Gibb,0.780952380952,5


In [90]:
knn_word_model=graphlab.nearest_neighbors.create(people,features=['word_count'],distance='cosine',label='name')

Starting brute force nearest neighbors model training.

In [91]:
knn_tfidf_model=graphlab.nearest_neighbors.create(people,features=['tfidf'],distance='cosine',label='name')

Starting brute force nearest neighbors model training.

In [92]:
knn_word_model.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.021ms      |

| Done         |         | 100         | 296.189ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


In [93]:
knn_tfidf_model.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.028ms      |

| Done         |         | 100         | 314.838ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [94]:
knn_word_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 8.023ms      |

| Done         |         | 100         | 284.757ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


In [95]:
knn_tfidf_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.025ms      |

| Done         |         | 100         | 308.212ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5
